In [2]:
import wandb
import random
from datetime import date, datetime as dt, timedelta
import datetime
import pandas as pd
from typing import Tuple
from warnings import simplefilter
import click
from dateutil.relativedelta import relativedelta  # type: ignore
import datetime  # type: ignore
import pandas as pd  # type: ignore
from loguru import logger

import torch
import torch.nn.functional as F
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import SAGEConv, to_hetero
import json
import pandas as pd
from datetime import datetime as dt
import os

from train_test_split import *
from evaluate import *
from constants import *
from model import *

In [3]:
wandb.login()


wandb: Currently logged in as: yhchan0918. Use `wandb login --relogin` to force relogin


True

In [4]:
# Load data
directory = "/Users/yhchan/Downloads/FYP/data/processed"


reviews = pd.read_parquet(f"{directory}/reviews_with_interactions.parquet")
listings = pd.read_parquet(f"{directory}/listings_with_interactions.parquet")
reviewers = pd.read_parquet(f"{directory}/reviewers_with_interactions.parquet")



In [8]:
iteration = 0
start_date = dt.strptime("2021-10-24", "%Y-%m-%d").date()
if start_date == dt.strptime(MAX_START_DATE, "%Y-%m-%d").date():
    raise Exception("Stop Simulation")
end_date, nxt_start_date = split_date_by_period_months(start_date, TOTAL_MONTHS_PER_ITERATION)

logger.info("Start of Retraining")
print(iteration)
print(start_date, end_date)



# Prepare data and graph
(
    train_reviews,
    train_listings,
    train_reviewers,
    test_reviews,
    test_listings,
    test_reviewers,
) = main_train_val_test(
    reviews,
    listings,
    reviewers,
    start_date,
    end_date,
    10
)

2023-03-12 14:20:33.361 | INFO     | __main__:<module>:7 - Start of Retraining


0
2021-10-24 2022-10-23
408596


2023-03-12 14:20:34.434 | INFO     | train_test_split:main_train_val_test:136 - Split df into train and test portion


In [42]:
def filter_test_data_by_scenario(train_reviews, test_reviews, user_col, scenario_type):
    if scenario_type == "cold_start":
        train_reviewers = list(train_reviews[user_col].unique())
        return test_reviews[~test_reviews[user_col].isin(train_reviewers)]

def get_nunique(df, col):
    return (df[col].nunique())


In [43]:
# start_date = dt.strptime(SIMULATION_START_DATE, '%Y-%m-%d').date()
# end_date, nxt_start_date = split_date_by_period_months(start_date, TOTAL_MONTHS_PER_ITERATION)


# (
# train_reviews,
# train_listings,
# train_reviewers,
# test_reviews,
# test_listings,
# test_reviewers,
# ) = main_train_val_test(reviews, listings, reviewers, start_date, end_date, 10)

# cold_start_test_reviews = filter_test_data_by_scenario(train_reviews, test_reviews, "reviewer_id", "cold_start")

In [44]:
iteration = 0
start_date = dt.strptime(SIMULATION_START_DATE, '%Y-%m-%d').date()
while True:
    print(iteration)
    end_date, nxt_start_date = split_date_by_period_months(start_date, TOTAL_MONTHS_PER_ITERATION)
    print(start_date, end_date)
    
    
    (
    train_reviews,
    train_listings,
    train_reviewers,
    test_reviews,
    test_listings,
    test_reviewers,
    ) = main_train_val_test(reviews, listings, reviewers, start_date, end_date, 10)
    
    cold_start_test_reviews = filter_test_data_by_scenario(train_reviews, test_reviews, "reviewer_id", "cold_start")
    
    data_dict = {
        "num_reviews": len(train_reviews) + len(test_reviews),
        "num_train_reviews":len(train_reviews),
        "num_test_reviews":len(test_reviews),
        "num_cold_start_test_reviews": len(cold_start_test_reviews),
        "num_unique_cold_start_test_listings":  get_nunique(cold_start_test_reviews, 'listing_id'),
        "num_unique_cold_start_test_reviewers":  get_nunique(cold_start_test_reviews, 'reviewer_id'),
        "num_unique_train_listings": get_nunique(train_listings, 'listing_id'),
        "num_unique_test_listings": get_nunique(test_listings, 'listing_id'),
        "num_unique_train_reviewers": get_nunique(train_reviewers, 'reviewer_id'),
        "num_unique_test_reviewers": get_nunique(test_reviewers, 'reviewer_id'),
    }
    
    print(data_dict)
    
    if start_date == dt.strptime(MAX_START_DATE, '%Y-%m-%d').date():
        break
    else:   
        start_date = nxt_start_date
        iteration += 1

2023-03-12 16:34:21.809 | INFO     | train_test_split:main_train_val_test:136 - Split df into train and test portion


0
2015-10-24 2016-10-23
53610
{'num_reviews': 53610, 'num_train_reviews': 40500, 'num_test_reviews': 13110, 'num_cold_start_test_reviews': 12980, 'num_unique_cold_start_test_listings': 2320, 'num_unique_cold_start_test_reviewers': 12830, 'num_unique_train_listings': 2706, 'num_unique_test_listings': 2331, 'num_unique_train_reviewers': 39828, 'num_unique_test_reviewers': 12952}
1
2016-10-24 2017-10-23
103877


2023-03-12 16:34:22.318 | INFO     | train_test_split:main_train_val_test:136 - Split df into train and test portion


{'num_reviews': 103877, 'num_train_reviews': 81540, 'num_test_reviews': 22337, 'num_cold_start_test_reviews': 22053, 'num_unique_cold_start_test_listings': 3748, 'num_unique_cold_start_test_reviewers': 21775, 'num_unique_train_listings': 4509, 'num_unique_test_listings': 3763, 'num_unique_train_reviewers': 79927, 'num_unique_test_reviewers': 22046}
2
2017-10-24 2018-10-23
160543


2023-03-12 16:34:22.874 | INFO     | train_test_split:main_train_val_test:136 - Split df into train and test portion


...

2023-03-12 16:34:25.015 | INFO     | train_test_split:main_train_val_test:136 - Split df into train and test portion


{'num_reviews': 239538, 'num_train_reviews': 184653, 'num_test_reviews': 54885, 'num_cold_start_test_reviews': 53548, 'num_unique_cold_start_test_listings': 9005, 'num_unique_cold_start_test_reviewers': 52840, 'num_unique_train_listings': 11274, 'num_unique_test_listings': 9079, 'num_unique_train_reviewers': 179415, 'num_unique_test_reviewers': 54102}
6
2021-10-24 2022-10-23
408596


2023-03-12 16:34:25.992 | INFO     | train_test_split:main_train_val_test:136 - Split df into train and test portion


{'num_reviews': 408596, 'num_train_reviews': 334678, 'num_test_reviews': 73918, 'num_cold_start_test_reviews': 71775, 'num_unique_cold_start_test_listings': 14254, 'num_unique_cold_start_test_reviewers': 70416, 'num_unique_train_listings': 17229, 'num_unique_test_listings': 14380, 'num_unique_train_reviewers': 324135, 'num_unique_test_reviewers': 72447}
7
2022-10-24 2023-10-23
0


TypeError: '>=' not supported between instances of 'datetime.date' and 'float'

In [4]:

# Driver code
iteration = 0
start_date = dt.strptime(SIMULATION_START_DATE, '%Y-%m-%d').date()
while True:
    print(iteration)
    end_date, nxt_start_date = split_date_by_period_months(start_date, TOTAL_MONTHS_PER_ITERATION)
    print(start_date, end_date)
    wandb.init(
          project=PROJECT_NAME, 
          name=f"iteration_{iteration}", 
          config={
            "architecture": "Unsupervised GraphSAGE",
            "iteration": iteration,
            "start_date":start_date,
            "end_date":end_date,
            "learning_rate": 0.01,
            "hidden_channels": 32,
            "train_batch_size": 1024,
            "test_batch_size": 256,
            "epochs": 2,
            "train_num_neighbours":[10, 10],
            "test_num_neighbours":[-1],
            "train_split_period_months":10,
            "total_months_of_data": TOTAL_MONTHS_PER_ITERATION,
          })
    wandb.define_metric("train_loss", step_metric="epoch", summary="min")
    wandb.define_metric("test_loss", step_metric="epoch", summary="min")

    # Prepare data and graph
    (
        train_reviews,
        train_listings,
        train_reviewers,
        test_reviews,
        test_listings,
        test_reviewers,
    ) = main_train_val_test(reviews, listings, reviewers, start_date, end_date, wandb.config['train_split_period_months'])
    wandb.log({
        "num_reviews": len(train_reviews) + len(test_reviews),
        "num_train_reviews":len(train_reviews),
        "num_test_reviews":len(test_reviews),
        "num_listings":len(train_listings) + len(test_listings),
        "num_train_listings":len(train_listings),
        "num_test_listings":len(test_listings),
        "num_reviewers":len(train_reviewers) + len(test_reviewers),
        "num_train_reviewers":len(train_reviewers),
        "num_test_reviewers":len(test_reviewers),
    })
    train_reviews.to_csv('train/train_reviews.parquet', index=False)
    train_listings.to_csv('train/train_listings.parquet', index=False)
    train_reviewers.to_csv('train/train_reviewers.parquet', index=False)
    test_reviews.to_csv('test/test_reviews.parquet', index=False)
    test_listings.to_csv('test/test_listings.parquet', index=False)
    test_reviewers.to_csv('test/test_reviewers.parquet', index=False)

    dataset_art = wandb.Artifact(f"{start_date}_{end_date}_data_{iteration}", type='dataset')
    for dir in ["train", "test"]:
        dataset_art.add_dir(dir)
    wandb.log_artifact(dataset_art)
    test_listings2dict = get_entity2dict(test_listings, "listing_id")
    reverse_test_listings2dict = {k: v for v, k in test_listings2dict.items()}
    test_reviewers2dict = get_entity2dict(test_reviewers, "reviewer_id")
    reverse_test_reviewers2dict = {k: v for v, k in test_reviewers2dict.items()}
    data = build_heterograph(reviews, listings)
    train_data = build_heterograph(train_reviews, train_listings)
    test_data = build_heterograph(test_reviews, test_listings)
    print("Training Heterogenous Graph", train_data)
    print("Test Heterogenous Graph", test_data)

    # Modelling
    train_loader = prepare_data_loader(
        data=train_data, batch_size=wandb.config['train_batch_size'], num_neighbours=wandb.config['train_num_neighbours']
    )
    test_loader = prepare_data_loader(data=test_data, batch_size=wandb.config['test_batch_size'], num_neighbours=wandb.config["test_num_neighbours"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_data = train_data.to(device)
    model = Model(hidden_channels=wandb.config['hidden_channels'], data).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config['learning_rate'])

    def train():
        model.train(True)
        total_loss = 0
        # Why using mini-batch gradient descent
        # Update NN multiple times every epoch, Make more precise update to the parameters by calculating the average loss in each step
        # Reduce overall training time and num of required epochs for reaching convergence, computational efficiency
        for batch in train_loader:
            batch = batch.to(device)
            # Zero gradients for every batch
            optimizer.zero_grad()
            # Make predictions for this batch
            h = model(batch.x_dict, batch.edge_index_dict)
            h_src = h["user"][batch["user", "listing"].edge_label_index[0]]
            h_dst = h["listing"][batch["user", "listing"].edge_label_index[1]]
            pred = (h_src * h_dst).sum(dim=-1)
            # Compute the loss and its gradients
            loss = F.binary_cross_entropy_with_logits(pred, batch["user", "listing"].edge_label)
            loss.backward()
            # Adjust learning weights
            optimizer.step()
            total_loss += float(loss) * pred.size(0)

        train_loss = total_loss / train_data.num_nodes
        return train_loss

    @torch.no_grad()
    def test(model, test_data_loader):
        model.eval()
        total_loss = 0
        for batch in test_data_loader:
            batch = batch.to(device)
            # Make predictions for this batch
            h = model(batch.x_dict, batch.edge_index_dict)
            h_src = h["user"][batch["user", "listing"].edge_label_index[0]]
            h_dst = h["listing"][batch["user", "listing"].edge_label_index[1]]
            pred = (h_src * h_dst).sum(dim=-1)
            # Compute the loss and its gradients
            loss = F.binary_cross_entropy_with_logits(pred, batch["user", "listing"].edge_label)
            total_loss += float(loss) * pred.size(0)

        test_loss = total_loss / test_data.num_nodes
        return test_loss

    best_train_loss = float('inf')
    best_test_loss = float('inf')
    best_model_path = None
    best_model_art = None
    # Train and Evaluate Loss
    for epoch in range(1, wandb.config['epochs'] + 1):
        model_is_best = False
        train_loss = train()
        test_loss = test(model, test_loader)

        if (train_loss < best_train_loss):
            wandb.run.summary["best_train_loss"] = train_loss
            best_train_loss = train_loss

        if (test_loss < best_test_loss):
            wandb.run.summary["best_test_loss"] = test_loss
            best_test_loss = test_loss
            model_is_best = True

        metrics_dict = {
            "train_loss":train_loss,
            "test_loss":test_loss,
            "epoch":epoch,
        }
        wandb.log(metrics_dict)
        logger.info(f"Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f} ")

        model_path = f"./models/{epoch}_model_state_dict.pt"
        torch.save(model.state_dict(), model_path)
        model_art = wandb.Artifact(f"{MODEL_NAME}_iteration_{iteration}", type='model')
        model_art.add_file(model_path)
        wandb.log_artifact(model_art, aliases=[LATEST_TAG, ] if model_is_best else None)
        if model_is_best:
            best_model_art = model_art
            best_model_path = model_path

    production_model = None
    try:
        production_model_dir = wandb.use_artifact(f'{ENTITY_NAME}/{PROJECT_NAME}/{MODEL_REGISTRY_NAME}:{PRODUCTION_TAG}').download()
        production_model_path = f"{production_model_dir}/{os.listdir(production_model_dir)[0]}"
        production_model = load_model(production_model_path, wandb.config['hidden_channels'], data)
        logger.info('Production model found')
    except Exception as e:
        print(e)
        logger.info('No production model found')


    contender_model = load_model(best_model_path, wandb.config['hidden_channels'], data)
    hit_rate_by_contender_model = evaluate_model(contender_model, test_data)
    wandb.log({"contender_model_hit_rate": hit_rate_by_contender_model}) 

    if production_model == None:
        is_contender_model_better = True
    else:
        test_loss_by_production_model = test(production_model, test_loader)    
        hit_rate_by_production_model = evaluate_model(production_model, test_data)
        wandb.log({"production_model_test_loss":test_loss_by_production_model, "production_model_hit_rate": hit_rate_by_contender_model}) 
        is_contender_model_better = test_loss_by_production_model < best_test_loss and hit_rate_by_contender_model > hit_rate_by_production_model

    model_registry_art = wandb.Artifact(MODEL_REGISTRY_NAME, type='model_registry')
    model_registry_art.add_file(best_model_path)
    # should go through shadow deployment or A/B Testing first before promoting it to production
    wandb.log_artifact(model_registry_art, aliases=[PRODUCTION_TAG, LATEST_TAG] if is_contender_model_better else [LATEST_TAG])
    # Save code
    wandb.run.log_code(
     "./", include_fn=lambda path: path.endswith(".py") or path.endswith(".ipynb")
    )
    wandb.finish()
    
    if start_date == dt.strptime(MAX_START_DATE, '%Y-%m-%d').date():
        break
    else:   
        start_date = nxt_start_date
        iteration += 1




1
2015-10-24 2016-10-23


53610


2023-03-04 14:57:01.723 | INFO     | train_test_split:main_train_val_test:136 - Split df into train and test portion
wandb: Adding directory to artifact (./train)... Done. 0.1s
wandb: Adding directory to artifact (./test)... Done. 0.0s
/Users/yhchan/Downloads/FYP/CT_of_recommendation_system/ml_pipeline/ct_run/train_test_split.py:173: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525682339/work/torch/csrc/utils/tensor_numpy.cpp:205.)
  temp = torch.from_numpy(val).view(-1, 1).to(torch.float32)


Training Heterogenous Graph HeteroData(
  listing={ x=[2706, 159] },
  user={ x=[39828, 1] },
  (user, rates, listing)={ edge_index=[2, 40500] },
  (listing, rev_rates, user)={ edge_index=[2, 40500] }
)
Test Heterogenous Graph HeteroData(
  listing={ x=[2331, 159] },
  user={ x=[12952, 1] },
  (user, rates, listing)={ edge_index=[2, 13110] },
  (listing, rev_rates, user)={ edge_index=[2, 13110] }
)


2023-03-04 14:57:04.630 | INFO     | __main__:<module>:149 - Epoch: 001, Train Loss: 51390321.6874, Test Loss: 4107816.4783 
2023-03-04 14:57:05.871 | INFO     | __main__:<module>:149 - Epoch: 002, Train Loss: 2049263.2889, Test Loss: 4306045.5998 
2023-03-04 14:57:07.508 | INFO     | __main__:<module>:168 - No production model found


...

2023-03-04 14:58:46.482 | INFO     | train_test_split:main_train_val_test:136 - Split df into train and test portion
wandb: Adding directory to artifact (./train)... Done. 0.2s
wandb: Adding directory to artifact (./test)... Done. 0.1s


Training Heterogenous Graph HeteroData(
  listing={ x=[6756, 159] },
  user={ x=[122793, 1] },
  (user, rates, listing)={ edge_index=[2, 125395] },
  (listing, rev_rates, user)={ edge_index=[2, 125395] }
)
Test Heterogenous Graph HeteroData(
  listing={ x=[5636, 159] },
  user={ x=[34614, 1] },
  (user, rates, listing)={ edge_index=[2, 35148] },
  (listing, rev_rates, user)={ edge_index=[2, 35148] }
)


2023-03-04 14:58:53.339 | INFO     | __main__:<module>:149 - Epoch: 001, Train Loss: 393890358.0131, Test Loss: 14518573.8910 
2023-03-04 14:58:58.969 | INFO     | __main__:<module>:149 - Epoch: 002, Train Loss: 142787.1928, Test Loss: 5278424.0801 
wandb:   1 of 1 files downloaded.  


./artifacts/Unsupervised_GraphSAGE:v0/2_model_state_dict.pt


contender_model_hit_rate,▁
epoch,▁█
num_listings,▁
num_reviewers,▁
num_reviews,▁
num_test_listings,▁
num_test_reviewers,▁
num_test_reviews,▁
num_train_listings,▁
num_train_reviewers,▁
num_train_reviews,▁


In [11]:
wandb.finish()

In [21]:
0 > float('-inf')

True

In [20]:

# Fetch the iteration, end date of last run to determine iteration, start_date of current run
# if cant find then the iteration is 0 and the start date just use the SIMULATION_START_DATE variable

try:
    api = wandb.Api()
    runs = api.runs(f"{ENTITY_NAME}/{PROJECT_NAME}")
    if len(runs) == 0:
        raise Exception("No runs")
    last_iteration = float('-inf')
    end_date_of_last_iteration = None

    for run in runs: 
        config = {k: v for k,v in run.config.items()
             if not k.startswith('_')}
        run_name = run.name
        iteration_no = int(run_name.split('_')[1])
        if  iteration_no > last_iteration:
            last_iteration = iteration_no
            end_date_of_last_iteration = config['end_date']
                  
    current_iteration = last_iteration + 1
    current_start_date = dt.strptime(end_date_of_last_iteration, "%Y-%m-%d").date() + relativedelta(days=1)
except Exception as e:
    print(e)
    current_iteration = 0
    current_start_date = dt.strptime(SIMULATION_START_DATE, "%Y-%m-%d").date()              
    



              
current_iteration, current_start_date              

<Runs yhchan0918/CT_OF_AIRBNB_RECSYS>
No runs


(0, datetime.date(2015, 10, 24))

(0, datetime.date(2015, 10, 24))

In [8]:
# Launch 5 simulated experiments
total_runs = 5

for run in range(1, total_runs + 1):
  # 🐝 1️⃣ Start a new run to track this script
    wandb.init(
      # Set the project where this run will be logged
      project="my-awesome-project", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      "iteration": run
      })

    # This simple block simulates a training loop logging metrics
    epochs = 10
    offset = random.random() / 5
    for epoch in range(2, epochs):
        acc = 1 - 2 ** -epoch - random.random() / epoch - offset
        loss = 2 ** -epoch + random.random() / epoch + offset
        wandb.log({"acc": acc, "loss": loss})
    
    today = (date.today() + timedelta(days=run))
    hit_rate = random.random()
    wandb.log({"hit_rate": hit_rate, "date":today }) 

    # Mark the run as finished
    wandb.finish()




acc,▁▆▄█▇▇█▇
hit_rate,▁
loss,█▄▂▁▂▃▁▁
acc,0.75372
date,2023-02-26
hit_rate,0.86809
loss,0.18551


acc,▁▅▆█▇██▇
hit_rate,▁
loss,█▄▇▃▄▃▁▂
acc,0.7306
date,2023-02-27
hit_rate,0.94817
loss,0.22256


acc,▁▅▃▄▇█▇█
hit_rate,▁
loss,█▃▂▃▂▂▂▁
acc,0.84853
date,2023-02-28
hit_rate,0.80284
loss,0.16238


acc,▁▂▂▅▅▇█▆
hit_rate,▁
loss,█▅▅▅▃▁▂▂
acc,0.70462
date,2023-03-01
hit_rate,0.8891
loss,0.25627


acc,▁▄▄▅▆███
hit_rate,▁
loss,█▄▃▃▁▂▁▁
acc,0.95688
date,2023-03-02
hit_rate,0.15433
loss,0.03116
